In [5]:
import json
import os

nums_agents = 20

run_data_folders = "real-experiment-runs/"

# cound number of folders in run_data_folders
num_folders = 0
for folder in os.listdir(run_data_folders):
    if os.path.isdir(run_data_folders + folder):
        num_folders += 1

num_folders = 31

for run_number in range(1, num_folders + 1):
    if run_number in [28, 29]:
        continue
    if run_number in [22, 23, 24]:
        nums_agents = 24
    else:
        nums_agents = 20
    run_subdirectory = run_data_folders + "run-" + str(run_number) + "/"

    round_data = {}

    for agent in range(1, nums_agents + 1):
        # open agent history file
        agent_file = run_subdirectory + "bot-" + str(agent) + "-history.json"
        with open(agent_file) as f:
            agent_history = json.load(f)
            for round in agent_history:
                round_num = round["round_num"]
                if round_num not in round_data:
                    round_data[round_num] = []
                # check if agent id is already in round data (recent addition) - key is agent_id
                if "agent_id" not in round:
                    round["agent_id"] = agent
                round_data[round_num].append(round)

    # Update keys to be of the form "Round {X - 3}" where X is the round number for X > 3 and "Practice Round X" for X <= 3
    round_data_keys = list(round_data.keys())
    for round_num in round_data_keys:
        # continue if round_num is not an integer
        if not isinstance(round_num, int):
            continue
        if round_num <= 3:
            round_data["Practice Round " + str(round_num)] = round_data.pop(round_num)
        else:
            round_data["Round " + str(round_num - 3)] = round_data.pop(round_num)
    # Write the data to a file
    with open(run_subdirectory + "all-rounds-compressed.json", "w") as f:
        json.dump(round_data, f, indent=4)

    # Make a human readable version of the plans, insights, and thoughts and observations
    # use the existing round_data
    output_file = run_subdirectory + "all-human-readable-insights.txt"
    with open(output_file, "w") as f:
        for round_key in round_data.keys():
            f.write(f"=== 🌀 Round: {round_key} 🌀 ===\n")
            for agent_round in round_data[round_key]:
                # agent id
                agent_id = agent_round["agent_id"]
                # plans
                plans = agent_round.get("plan", "🚫 No Plan Written")
                # insights
                insights = agent_round.get("insight", "🚫 No Insight Written")
                # thoughts and observations
                thoughts_observations = agent_round.get("thought_observation", "🚫 No Thought/Observation Written")
                
                # Write to file in a human-readable format
                f.write(f"🔹 Agent ID: {agent_id}\n")
                f.write(f"🔸 Plans: {plans}\n")
                f.write(f"🔸 Insights: {insights}\n")
                f.write(f"🔸 Thoughts and Observations: {thoughts_observations}\n")
                f.write("\n")  # Add a newline for better readability